# Importing libraries

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report
import pathlib

import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image

C:\Users\hunda\Anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# data reading and splitting

In [2]:
image_dir = Path('C:/Users/hunda/OneDrive/Desktop/liver1')
#image_dir2 = Path('C:/Users/hunda/OneDrive/Desktop/heart-test')
#data_dir = pathlib.Path(image_dir)

In [3]:
from keras.preprocessing.image import ImageDataGenerator

trdata = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


traindata = trdata.flow_from_directory(directory=image_dir,target_size=(224,224),
    batch_size = 16, subset="training")
valdata = trdata.flow_from_directory(directory=image_dir,target_size=(224,224),
    batch_size = 16,subset="validation")


Found 1419 images belonging to 2 classes.
Found 353 images belonging to 2 classes.


# implementing vgg16

In [4]:
vgg = VGG16(input_shape= [224,224]+[3] ,include_top=False, weights='imagenet')
vgg.trainable= False
#DEFINING THE VGG16

In [5]:
# for layers in (vgg.layers)[:14]:
#     print(layers)
#     layers.trainable = False
#freezing some layers for tunning the model

In [6]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

# fittting the model and defining the layers

In [7]:
#vgg.trainable = False ## Not trainable weights
# from tensorflow.keras.applications.vgg16 import preprocess_input
# train_ds = preprocess_input(train_ds) 
# val_ds = preprocess_input(val_ds)

In [8]:
# normalization_layer = tf.keras.layers.Rescaling(1./255)
# normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# normalized_ds2 = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [9]:
# from tensorflow.keras import layers, models

# # flatten_layer = layers.Flatten()
# # dense_layer_1 = layers.Dense(50, activation='relu')
# # dense_layer_2 = layers.Dense(20, activation='relu')
# # prediction_layer = layers.Dense(2, activation='softmax')
# num_classes = 2
# flatten_layer = layers.Flatten()
# model = tf.keras.Sequential([
#   tf.keras.layers.Rescaling(1./255),flatten_layer,
#   tf.keras.layers.Conv2D(128, 3, activation='relu'),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Conv2D(64, 3, activation='relu'),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Conv2D(32, 3, activation='relu'),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Flatten(),
#   #tf.keras.layers.Dense(16, activation='softmax'),
#   tf.keras.layers.Dense(num_classes)
# ])



# # model = models.Sequential([
# #     vgg,
# #     flatten_layer,
# #     dense_layer_1,
# #     dense_layer_2,
# #     prediction_layer
# # ])

In [10]:

from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(64, activation='relu')
dense_layer_2 = layers.Dense(32, activation='relu')
dense_layer_3 = layers.Dense(16, activation='relu')
prediction_layer = layers.Dense(2, activation='softmax')


model = models.Sequential([
    vgg,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    dense_layer_3,
    prediction_layer
])

In [11]:
model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x000001C304C8E7F0>>

In [12]:
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
lr = 1e-4
optimizer = Adam(lr)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)


filepath='C:/Users/hunda/OneDrive/Desktop/liver-test'
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)
callbacks_list = [checkpoint, es]

model.fit(traindata, validation_data=valdata, epochs=1, callbacks=callbacks_list)

89/89 [==============================] - ETA: 0s - loss: 0.4118 - accuracy: 0.7984
Epoch 1: val_loss improved from inf to 0.38780, saving model to C:/Users/hunda/OneDrive/Desktop\liver-test


INFO:tensorflow:Assets written to: C:/Users/hunda/OneDrive/Desktop\liver-test\assets


INFO:tensorflow:Assets written to: C:/Users/hunda/OneDrive/Desktop\liver-test\assets


89/89 [==============================] - 220s 2s/step - loss: 0.4118 - accuracy: 0.7984 - val_loss: 0.3878 - val_accuracy: 0.8555


# accuracy

In [14]:
results = model.evaluate(valdata, verbose = 0)
print("  test loss: {:.5f}".format(results[0]))
print("test accuracy: {:.2f}%".format(results[1]*100))

  test loss: 0.39421
test accuracy: 85.84%


# plotting

In [21]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [20]:
pip install pydot 

Note: you may need to restart the kernel to use updated packages.
